In [2]:
import os
import pickle
import numpy as np
import pandas as pd
import gc

from sklearn.utils import Bunch

from proxbias import utils
from proxbias.cpg_processing import loading as cpgl
from proxbias.cpg_processing import transform as cpgt
from proxbias.arm_centering import build_arm_centering_df, perform_arm_centering

# Load cpg0016 data
The metadata is stored locally because there isn't an API for downloading it. But features are pulled from the S3 bucket provided by the JUMP consortium.

In [4]:
me = cpgl.load_cpg_crispr_well_metadata()
# Sometimes this fails on the first run because of some S3 weirdness, but works the second time
# so we just try twice here.
try:
    fe = cpgl.load_feature_data(me)
except ValueError:
    fe = cpgl.load_feature_data(me)
data = cpgl.build_combined_data(me, fe)
del me, fe
gc.collect()

130

# Process the cpg0016 data using PCA

In [5]:
data = cpgt.preprocess_data(data)
data_t = cpgt.transform_data(data)

In [6]:
n_feats = data_t.shape[1] - 1
print(f"Data went from {data.shape[1]} to {n_feats} features. Keeping 98% of the variance")

Data went from 3677 to 155 features. Keeping 98% of the variance


# Apply arm-centering correction 

- Add in the chromosome annotations 
- Add in expression annotations (zfpkm values from Recursion RNAseq in U2OS cells)
- Subtract off the mean of unexpressed genes per chromosome arm

### Add in chromosome information

In [7]:
gene_dict, chrom_dict, band_dict = utils.chromosome_info.get_chromosome_info_as_dicts()
genes = list(gene_dict.keys())

In [8]:
# Subset to genes with chrom information from hg38
idx = data_t.query(f"Metadata_Symbol.isin({genes})").index
print(f"Full data has {data_t.shape[0]} genes, {len(idx)} of which are in hg38 annotations")
data_t = data_t.rename({"Metadata_Symbol": "gene"}, axis=1)
data_t = data_t.loc[idx].reset_index(drop=True)

# Add in chromomsome information
data_t["chromosome"] = data_t.gene.apply(lambda x: gene_dict[x]["chrom"] if x in gene_dict else "no info")
data_t["chr_idx"] = data_t.gene.apply(lambda x: gene_dict[x]["chrom_int"] if x in gene_dict else "no info")
data_t["chromosome_arm"] = data_t.gene.apply(lambda x: gene_dict[x]["arm"] if x in gene_dict else "no info")
data_t["gene_bp"] = data_t.gene.apply(lambda x: gene_dict[x]["start"] if x in gene_dict else "no info")

Full data has 7977 genes, 7825 of which are in hg38 annotations


### Add in expression data

In [9]:
u2os_exp = pd.read_csv("../data/u2os.csv", index_col="gene")

In [10]:
data_t = data_t.assign(zfpkm=data_t.gene.map(u2os_exp.zfpkm.to_dict()))

cols = ["gene", "chromosome", "chr_idx", "chromosome_arm", "gene_bp", "zfpkm"] + list(range(0, n_feats))
data_t = data_t.loc[:, cols]
data_t = data_t.sort_values(["chr_idx", "gene_bp"]).reset_index(drop=True)

In [11]:
data_t.head()

,gene,chromosome,chr_idx,chromosome_arm,gene_bp,zfpkm,1,2,3,4,...,145,146,147,148,149,150,151,152,153,154
0,HES4,chr1,1,chr1p,998963,-4.060299,-0.293430,0.182932,-0.505629,-0.538439,...,-0.882167,0.538489,-0.030721,0.099025,0.001580,0.690363,0.290000,-0.866354,0.216838,-0.000428
1,RNF223,chr1,1,chr1p,1070966,-3.780112,0.507977,0.062129,-1.382791,-0.324926,...,-0.182823,0.256507,0.053999,0.274025,0.249013,0.265098,0.079749,-0.102031,-0.465650,0.289305
2,TNFRSF4,chr1,1,chr1p,1211339,-2.613715,0.670106,0.299331,0.097489,0.291351,...,-0.697150,0.584254,0.044783,-0.198490,0.593804,0.383615,-0.388746,-0.199009,-0.787844,0.007407
3,B3GALT6,chr1,1,chr1p,1232236,-0.488317,-0.798343,0.086292,-0.034710,-0.250217,...,-1.045757,-0.294913,0.022396,0.278259,0.101221,0.520363,0.453335,0.094467,-0.009745,0.112679
4,UBE2J2,chr1,1,chr1p,1253911,0.723001,-0.264670,0.689511,0.307939,-0.239155,...,0.444107,0.080091,0.791465,-0.468546,0.258780,1.607342,0.433997,0.362857,-0.066953,0.117223


### Prox-bias correction: subtract off the mean of unexpressed genes per chromosome arm

In [12]:
cols = ["gene", "chromosome", "chr_idx", "chromosome_arm", "gene_bp", "zfpkm"]
arm_df = build_arm_centering_df(data=data_t, metadata_cols=cols)
data_pbc = perform_arm_centering(data=data_t, metadata_cols=cols, arm_centering_df=arm_df)

# Combine corrected and un-corrected datasets into a dictionary of bunch objects and write out pickled files

In [13]:
data_pbc.head()

,gene,chromosome,chr_idx,chromosome_arm,gene_bp,zfpkm,1,2,3,4,...,145,146,147,148,149,150,151,152,153,154
0,HES4,chr1,1,chr1p,998963,-4.060299,-0.114877,0.279856,-0.264996,0.006656,...,-0.702039,0.384718,-0.065364,-0.032908,0.398196,0.422840,0.297409,-0.878708,0.466026,0.029106
1,RNF223,chr1,1,chr1p,1070966,-3.780112,0.686530,0.159053,-1.142159,0.220169,...,-0.002694,0.102735,0.019356,0.142092,0.645630,-0.002425,0.087159,-0.114385,-0.216461,0.318839
2,TNFRSF4,chr1,1,chr1p,1211339,-2.613715,0.848659,0.396255,0.338121,0.836446,...,-0.517021,0.430483,0.010141,-0.330424,0.990421,0.116092,-0.381337,-0.211363,-0.538656,0.036941
3,B3GALT6,chr1,1,chr1p,1232236,-0.488317,-0.619790,0.183216,0.205923,0.294879,...,-0.865629,-0.448684,-0.012247,0.146325,0.497838,0.252840,0.460744,0.082113,0.239443,0.142212
4,UBE2J2,chr1,1,chr1p,1253911,0.723001,-0.086117,0.786435,0.548572,0.305941,...,0.624236,-0.073681,0.756822,-0.600479,0.655396,1.339819,0.441406,0.350503,0.182235,0.146757


In [14]:
data_dict = {
    "JUMP-CP PCAW": Bunch(metadata=data_t.loc[:, cols], features=data_t.loc[:, list(range(0, n_feats))]),
    "JUMP-CP PCAW Arm-Centered": Bunch(
        metadata=data_pbc.loc[:, cols], features=data_pbc.loc[:, list(range(0, n_feats))]
    ),
}

In [15]:
path = "../../scratch"
if not os.path.exists(path):
    os.mkdir(path)

with open(os.path.join(path, "cpg_data_dict.pkl"), "wb") as outfile:
    pickle.dump(data_dict, outfile)